In [ ]:
include("utilities.jl");

In [ ]:
# Mask which defines the observations that can be made
V = [1 1 1; 1 0 1; 1 1 1];

In [ ]:
# Specify for which γ, how many experiments are done; also specify the size
k = 10^3;
n_averages = 100;
sidelength = 5
w = sidelength;
h = sidelength;
ε = 3*10^-5;
γRange = [1 - ε*k + j*ε for j in 1:(k-1)];

In [ ]:
# Read mazes that should be solved
mazes = [zeros(2*sidelength+1, 2*sidelength+1) for j in 1:n_averages];

test = readdlm("mazes/mazes$sidelength.csv", ',', Float64)
test = reshape(test, 100, 2*sidelength+1, 2*sidelength+1)
for j in 1:n_averages
    mazes[j] = test[j,:,:]
end

In [ ]:
# Allocate memory for the computations with fixed discount
timesBCP = zeros(length(γRange), n_averages);
rewardsBCP = zeros(length(γRange), n_averages);
statusBCP = [];

In [ ]:
@elapsed for j in 1:n_averages
    M = mazes[j];
    states = listOfStates(M);
    goal = rand(states);
    α = transitionKernel(M,A,goal);
    β = observationKernel(M,V);
    r = instReward(M,A,goal);
    μ = initialDistribution(M);
    for i in 1:length(γRange)
        γ = γRange[i]
        # DPO
        πBCP, modelBCP = BCP(α, β, r, μ, γ, 0)
        rewardsBCP[i,j] = R(πBCP, α, β, γ, μ, r)
        timesBCP[i,j] = solve_time(modelBCP)
        statusBCP = vcat(statusBCP, raw_status(modelBCP))
    end
end

In [ ]:
writedlm("data/BCPSizeFixedRewards.csv", rewardsBCP, ',')
writedlm("data/BCPSizeFixedTimes.csv", timesBCP, ',')
writedlm("data/BCPSizeFixedStatus.csv", statusBCP, ',')